In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 47.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=14a9cf117b7d191572b973c31b9f9ed9d1426768fd6a8c1212f0ff613ab94432
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [11]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from sklearn.metrics import accuracy_score

spark = SparkSession.builder.appName('mobileprice_lr').getOrCreate()
df = spark.read.csv('dataset.csv', header = True, inferSchema = True)

numericCols = ['battery_power', 'clock_speed', 'dual_sim', 'fc', 'four_g', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores','pc', 'px_height', 'px_width','ram','sc_h','sc_w','talk_time','three_g','touch_screen','wifi']

featurizationPipeline = Pipeline(stages = [VectorAssembler(inputCols=numericCols, outputCol="feature_vector")])

featurizationPipelineModel = featurizationPipeline.fit(df)
df = featurizationPipelineModel.transform(df)

train, test = df.randomSplit([0.8, 0.2], seed = 2018)

lr = LogisticRegression(featuresCol = 'feature_vector', labelCol = 'price_range', maxIter=10)

lrModel = lr.fit(train)
predictions = lrModel.transform(test)

true_labels=predictions.select('price_range')
lr_predictions=predictions.select('prediction')

accuracy = accuracy_score(true_labels.toPandas(), lr_predictions.toPandas())
print("Logistic Regression Accuracy =",accuracy*100,"%")

Logistic Regression Accuracy = 96.40102827763496 %
